# **Pandas Tutorial**

This tutorial uses SQL as a way to teach Pandas commom operations. Do skim through this [Pandas official SQL comparision](https://pandas.pydata.org/pandas-docs/stable/getting_started/comparison/comparison_with_sql.html) and this [medium post](https://medium.com/jbennetcodes/how-to-rewrite-your-sql-queries-in-pandas-and-more-149d341fc53e) before starting this tutorial.

---

In [0]:
# Importing the pandas package
import pandas as pd
import numpy as np

# Loading the datasets as Pandas Dataframe
airports = pd.read_csv('https://ourairports.com/data/airports.csv')
airport_freq = pd.read_csv('https://ourairports.com/data/airport-frequencies.csv')
runways = pd.read_csv('https://ourairports.com/data/runways.csv')

###**Selecting columns**
We will learn of how to **select columns of a dataframe**. Refer [this article](https://www.geeksforgeeks.org/how-to-select-multiple-columns-in-a-pandas-dataframe/) for more details.

iloc and loc can get very confusing sometimes. In simple fashion, loc is label based and iloc is index based. Refer [this analytics vidhya article](https://www.analyticsvidhya.com/blog/2020/02/loc-iloc-pandas/) for details.

In [0]:
# SELECT * FROM airports
airports

# SELECT * FROM airports limit 3
temp = airports.head(3)
print(temp)

temp = airports.loc[0:2,:] # 3 rows (0,1,2) and all columns
print(temp)

temp = airports.iloc[0:3,:] # Notice the 3 is the range for rows, in iloc last parameter to range is not inclusive
print(temp)

# SELECT ident, type, name FROM airport 
temp = airports[['ident', 'type', 'name']]
print(temp)

temp = airports[airports.columns[1:4]] # airports.column is an attribute of type index and hence can be used directly
print(temp)

temp = airports.loc[:, ['ident', 'type', 'name']]
temp = airports.loc[:, 'ident':'name'] # taking in the range of columns from ident to name
print (temp)

temp = airports.iloc[:, 1:4] # using iloc, first value is inclusive and last exclusive (hence started with 1 and ended with 4 to take in 3 columns)
temp = airports.iloc[:, [1,2,3]] # To exemplify, 1:4 is same as passing in an array of 1, 2, 3
print (temp)

# SELECT type FROM airport limit 6
temp = airports['type'].head(6)
temp = airports.type.head(6)
print(temp)

# SELECT ident, type, name FROM airport limit 3
temp = airports[0:3][['ident', 'type', 'name']]
temp = airports.loc[0:2, ['ident', 'type', 'name']]
temp = airports.iloc[0:3, 1:4]
print(temp)


### **Filtering and sorting**

**Filtering and sorting rows** of a dataframe. Do remember that string matching taking in a regex value. Refer to [this article](https://stackoverflow.com/questions/22291565/pandas-text-matching-like-sqls-like) for more details.

In [0]:
# Filter out all rows with missing values
airports.dropna()
        
# Filter out all columns with missing value
airports.dropna(axis=1)

# SELECT * FROM airports WHERE iso_region is not null
airports[airports.iso_region.notna()]

# SELECT ident, name, municipality FROM airports WHERE iso_region = 'US-CA' and type = 'large_airport'
temp = airports[(airports.iso_region == 'US-CA') & (airports.type == 'large_airport')] [['ident', 'name', 'municipality']] # using default dataframe filters
temp = airports.loc[(airports.iso_region == 'US-CA') & (airports.type == 'large_airport'), ['ident', 'name', 'municipality']] # using loc
temp = airports.iloc[ ((airports.iso_region == 'US-CA') & (airports.type == 'large_airport')).values, [1,3,10]] #iloc doesn't take in a boolean series, hence with .values we create is as a numpy array
print(temp)

# SELECT ident, name, municipality FROM airports WHERE iso_region = 'US-CA' and type = 'large_airport' ORDER BY name DESC, municipality 
temp = airports[(airports.iso_region == 'US-CA') & (airports.type == 'large_airport')] [['ident', 'name', 'municipality']]
temp = temp.sort_values(['name', 'municipality'], ascending= [False, True])
print(temp)

# SELECT * FROM airports WHERE type not in ('heliport', 'balloonport')
airports[~airports.type.isin(['heliport', 'balloonport'])]

# SELECT * FROM airports WHERE type in ('heliport', 'balloonport')
airports[airports.type.isin(['heliport', 'balloonport'])]

# SELECT ident, type, name FROM airports WHERE type like '%airport%' and (iso_region = 'US-AZ' OR iso_region = 'CN-21')
airports[(airports.type.str.contains('.*airport.*'))& ((airports.iso_region == 'US-AZ')|(airports.iso_region == 'CN-21'))] [['ident', 'type', 'name']] # Contains takes in regex
airports.loc[(airports.type.str.contains('.*airport.*'))& ((airports.iso_region == 'US-AZ')|(airports.iso_region == 'CN-21')), ['ident', 'type', 'name']] 
airports.iloc[((airports.type.str.contains('.*airport.*'))& ((airports.iso_region == 'US-AZ')|(airports.iso_region == 'CN-21'))).values, 1:4] 
airports.query('type.str.contains(".*airport.*") and iso_region == "US-AZ" or iso_region == "CN-12" ', inplace = False, engine='python')[['ident', 'type', 'name']] # DONT USE QUERY with python engine - gets slow on big data. Also its confusing to use 'and' and 'or' togeher here.

idx = np.where(airports.type.str.contains('.*airport.*') & ((airports.iso_region == 'US-AZ')|(airports.iso_region == 'CN-21'))) # np.where returns the list of indexes that match criteria which is passed into the loc function
airports.loc[idx][['ident', 'type', 'name']]

### **Data aggregation**

Group by will return a pivot table kind of dataframe upon execution, to get a SQL style dataframe where the aggregations are new columns to the dataframe, we would need do to do step. Refer [this answer](https://stackoverflow.com/questions/53781634/aggregation-in-pandas) for very good insights into group by.
1.   [Unstack the pivot](https://stackoverflow.com/questions/41658498/in-pandas-after-groupby-the-grouped-column-is-gone) by passing in as_index = False into the groupby function
2.   Merge the multiple layered columns of the data (called indexes)

For SQL partition by clause, pandas provides the rank method. A good reference guide is [here](https://dfrieds.com/data-analysis/rank-method-python-pandas.html).



In [0]:
# SELECT count(*), avg(lenght_ft), max(length_ft), median(width_ft) FROM runways
temp = runways.agg({'length_ft': [min, max], 'width_ft': max})
print(temp)

# SELECT surface, lighted, count(*), avg(lenght_ft), max(length_ft), median(width_ft) FROM runways GROUP BY surface, lighted
temp = runways.groupby(['surface', 'lighted'], as_index= False).agg({'length_ft': [np.size, np.mean, np.max], 'width_ft': np.max})
temp.columns = pd.Index([e[0] + e[1] for e in temp.columns.tolist()]) # Need to update the columns else it returns a pivot table style output (wide to long maybe another option - reshape)
temp
print(temp)

# PREFERRED way of doing the same above SQL style query.
temp = runways.groupby(['surface', 'lighted']).agg(length_ft_cnt = ('length_ft', 'size'), 
                                                   length_ft_mean = ('length_ft', 'mean'), 
                                                   length_ft_max = ('length_ft', 'max'), 
                                                   width_ft_max = ('width_ft', 'max')
                                                  ).reset_index()
temp.head()

You can also write your own custom functions for manupulating aggregations, a good reference is [here](https://stackoverflow.com/questions/44348426/pandas-groupby-custom-function-to-each-series). 

Doing **SQL sytle window functions** include: ROW_NUMBER(), RANK(), DENSE_RANK() and NTILE(). Refer to [this article](https://dfrieds.com/data-analysis/rank-method-python-pandas.html) for more details. 

In [0]:
# create a sample dataframe
data = pd.DataFrame({
    'age' :     [ 10, 22, 13, 21, 12, 11, 17, 11],
    'section' : [ 'A', 'B', 'C', 'B', 'B', 'A', 'A', 'B'],
    'city' :    [ 'Delhi', 'Delhi', 'Mumbai', 'Delhi', 'Mumbai', 'Delhi', 'Mumbai', 'Delhi'],
    'gender' :  [ 'M', 'F', 'F', 'M', 'M', 'M', 'F', 'M'],
    'favourite_color' : [ 'red', np.NAN, 'yellow', np.NAN, 'black', 'green', 'red', 'red']
})
data

# create a sample dataframe
def mean_gap(a):
  """Returns a float
  if a = [1,3,7] mean_gap(a) will give me (abs(3-1) + abs(7-3))/2 = 3.0"""

    b = []
    a = np.asarray(a) # important to convert series to numpy array
    a = np.sort(a)    # sorting maybe necessary for computations that require a specfic ordering of values to get right results
    for i in range(0, len(a)-1):
        b.append(abs(a[i+1]-a[i]))
    return np.mean(b) 

temp = data.groupby(['city', 'gender']).agg(avg_age = ('age', 'mean'),
                                            gap_age = ('age', mean_gap)).reset_index() # used a custom function
print(temp)
data.sort_values(['city', 'gender', 'age'])

# ROW_NUMBER()
data['age_rank'] = data.groupby(['section', 'city'])['age'].rank(method = 'first')
data['age_rank'] = data.groupby(['city', 'gender'])['age'].rank(method = 'first', ascending = False)
data.sort_values(['city','gender', 'age'])

# RANK()
data['age_rank'] = data.groupby(['city', 'gender'])['age'].rank(method = 'min', ascending = False)
data.sort_values(['city','gender', 'age'])

# DENSE_RANK()
data['age_rank'] = data.groupby(['city', 'gender'])['age'].rank(method = 'dense', ascending = False)
data.sort_values(['city','gender', 'age'])

# NTILE()
data.groupby(['city', 'gender'])['age'].rank(pct = True, ascending = False)*100
data['age_ntile_city_gender'] = data.groupby(['city', 'gender'])['age'].rank(pct = True, ascending = False)*100 # See how the percentiles are reversed due to ascending
data.sort_values(['city','gender', 'age'])

data['age_percentile'] = data['age'].rank(pct = True)*100
print(data.sort_values('age'))

data.age.quantile(.5) # Get quantile of a specific column

### **JOINs and UNIONs** 

In [0]:
airports.head()
print(airports.columns)
airport_freq.head()

airport_freq.merge(airports[['id']], left_on='airport_ref', right_on='id', how='inner')[['airport_ident', 'type', 'description', 'frequency_mhz']]

data_new = pd.DataFrame({
    'city' :    [ 'Delhi'],
    'location': [ 'North']
})

# DOING JOINS
data_new.merge(data, left_on='city', right_on='city', how = 'outer') # Full outer join
data_new.merge(data, left_on='city', right_on='city', how = 'inner') # Inner join
data_new.merge(data, left_on='city', right_on='city', how = 'left') # Left join
data_new.merge(data, left_on='city', right_on='city', how = 'right') # Right join


# DOING UNIONS
data1 = pd.DataFrame({
    'age' :     [ 10, 22, 13, 21, 12, 11, 17, 11],
    'city' :    [ 'Delhi', 'Delhi', 'Mumbai', 'Delhi', 'Mumbai', 'Delhi', 'Mumbai', 'Delhi'],
})

data2 = pd.DataFrame({
    'age' :     [ 10, 22, 13, 18, 2, 11],
    'city' :    [ 'Delhi', 'Delhi', 'Mumbai', 'Delhi', 'Mumbai', 'Delhi'],
})

# UNION ALL
pd.concat([airports[airports.ident == 'KLAX'][['name', 'municipality']], airports[airports.ident == 'KLGB'][['name', 'municipality']]])
pd.concat([data1, data2]) # Pass in a list

# UNION
pd.concat([data1, data2]).drop_duplicates() 

### **Creating new columns**
Applying **custom functions** to rows and columns and creating new columns

In [104]:
# Using a simple ifelse logic to create flag columns
runways['Wide_runway'] = np.where(runways['width_ft']>=100, 'yes', 'no')

# Apply a custom function to every row in a pandas dataframe
# 1 meter = 3.28 foot
# Doing without any custom function 
runways['length_mtr'] = runways['length_ft'] / 3.28
runways['area_mtr'] = (runways['length_ft'] / 3.28) * (runways['width_ft'] / 3.28)

# Using .apply and custom functions
def feet_to_meter(value):
  return value / 3.28
runways['length_mtr'] = runways['length_ft'].apply(feet_to_meter)

def calculate_area(row):
    length = feet_to_meter(row['length_ft'])
    width  = feet_to_meter(row['width_ft'])
    return length * width

runways['area_mtr'] = runways.apply(calculate_area, axis = 1)

# Using .apply and customizing a function even more
def calculate_area1(row):
    length = feet_to_meter(row['length_ft'])
    width  = feet_to_meter(row['width_ft'])
    if (row['surface'] == 'ASPH-G') | (row['surface'] == 'GRASS'):
      length = 0
    return length * width

runways['area_mtr'] = runways.apply(calculate_area1, axis = 1)

runways

,id,airport_ref,airport_ident,length_ft,width_ft,surface,lighted,closed,le_ident,le_latitude_deg,le_longitude_deg,le_elevation_ft,le_heading_degT,le_displaced_threshold_ft,he_ident,he_latitude_deg,he_longitude_deg,he_elevation_ft,he_heading_degT,he_displaced_threshold_ft,length_mtr,area_mtr,Wide_runway
0,12,12,12,12.0,12.0,hoot,1,0,H1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.658537,13.384890,yes
1,255155,6524,00AK,2500.0,70.0,hoot,0,0,N,NaN,NaN,NaN,NaN,NaN,S,NaN,NaN,NaN,NaN,NaN,762.195122,16266.359310,yes
2,254165,6525,00AL,2300.0,200.0,hoot,0,0,01,NaN,NaN,NaN,NaN,NaN,19,NaN,NaN,NaN,NaN,NaN,701.219512,42757.287329,yes
3,270932,6526,00AR,40.0,40.0,hoot,0,0,H1,NaN,NaN,NaN,NaN,NaN,H1,NaN,NaN,NaN,NaN,NaN,12.195122,148.720999,yes
4,322128,322127,00AS,1450.0,60.0,hoot,0,0,1,NaN,NaN,NaN,NaN,NaN,19,NaN,NaN,NaN,NaN,NaN,442.073171,8086.704343,yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41528,327703,327451,ZYSQ,8202.0,148.0,NaN,0,0,04,NaN,NaN,NaN,NaN,NaN,22,NaN,NaN,NaN,NaN,NaN,2500.609756,112832.391434,yes
41529,235188,27242,ZYTL,10827.0,148.0,CON,1,0,10,38.9671,121.520,105.0,95.6,650.0,28,38.9642,121.558,85.0,275.6,320.0,3300.914634,148943.709102,yes
41530,235186,27243,ZYTX,10499.0,148.0,CON,1,0,06,41.6304,123.469,171.0,48.7,NaN,24,41.6493,123.498,197.0,228.7,NaN,3200.914634,144431.513980,yes
41531,235169,27244,ZYYJ,8530.0,148.0,CON,1,0,09,42.8811,129.436,623.0,81.7,NaN,27,42.8845,129.467,597.0,261.7,NaN,2600.609756,117344.586556,yes


### **Iterating through the dataframe** 
[This article](https://thispointer.com/pandas-6-different-ways-to-iterate-over-rows-in-a-dataframe-update-while-iterating-row-by-row/) shows various ways to use for loop on a dataframe. This will be very slow. This should be the last resort if custom functions in the split-apply-combine strategy is too hard to implement and this is a one time thing.

In [0]:
# Loop through rows of dataframe by index i.e. from 0 to number of rows
temp = runways.head()
for i in range(0, temp.shape[0]):
  rowSeries = temp.iloc[i] # get row contents as series using iloc{] and index position of row
  print(rowSeries.values) # print row contents
  temp['surface'].iloc[i] = 'hoot' # update the cell value in temp dataframe

temp.head()